**Garbage Segregation from Images using YOLO algorithm**



In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Get the count of Images in dataset**

In [ ]:
import os

# Define the base path for the dataset
base_path = '/content/drive/MyDrive/Garbage Segregation Dataset_new/'

# Define the categories
categories = ['Cloth', 'Glass', 'Plastic', 'Paper']

# Loop through each category and count the files in 'Images' and 'Labels' subfolders
for category in categories:
    category_path = os.path.join(base_path, category)
    images_path = os.path.join(category_path, 'Images')
    labels_path = os.path.join(category_path, 'Labels')

    # Count the number of files in each subfolder
    images_count = len([file for file in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, file))])
    labels_count = len([file for file in os.listdir(labels_path) if os.path.isfile(os.path.join(labels_path, file))])

    # Print the counts for the current category
    print(f"Category: {category}")
    print(f"  Number of files in 'Images' subfolder: {images_count}")
    print(f"  Number of files in 'Labels' subfolder: {labels_count}")
    print()


Category: Cloth
  Number of files in 'Images' subfolder: 764
  Number of files in 'Labels' subfolder: 764

Category: Glass
  Number of files in 'Images' subfolder: 750
  Number of files in 'Labels' subfolder: 750

Category: Plastic
  Number of files in 'Images' subfolder: 757
  Number of files in 'Labels' subfolder: 757

Category: Paper
  Number of files in 'Images' subfolder: 754
  Number of files in 'Labels' subfolder: 754



**Split the dataset**

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
base_path = "/content/drive/MyDrive/Garbage Segregation Dataset_new/"  # Current dataset with categories
output_path = "/content/drive/MyDrive/Dataset_Split_for_yolo/"           # Target dataset folder

# Categories in your dataset
categories = [ "Cloth", "Glass", "Paper","Plastic"]

# Create train, test, and val directories
os.makedirs(os.path.join(output_path, "train/images"), exist_ok=True)
os.makedirs(os.path.join(output_path, "train/labels"), exist_ok=True)
os.makedirs(os.path.join(output_path, "val/images"), exist_ok=True)
os.makedirs(os.path.join(output_path, "val/labels"), exist_ok=True)
os.makedirs(os.path.join(output_path, "test/images"), exist_ok=True)
os.makedirs(os.path.join(output_path, "test/labels"), exist_ok=True)

# Collect all images and labels
all_images = []
all_labels = []

for category in categories:
    image_folder = os.path.join(base_path, category, "Images")
    label_folder = os.path.join(base_path, category, "Labels")

    # Get all images in the folder
    images = sorted(os.listdir(image_folder))

    # Ensure corresponding label files exist
    labels = [f.rsplit(".", 1)[0] + ".txt" for f in images]
    images_with_labels = [
        img for img, lbl in zip(images, labels)
        if os.path.exists(os.path.join(label_folder, lbl))
    ]

    # Update image and label lists
    all_images.extend([os.path.join(image_folder, img) for img in images_with_labels])
    all_labels.extend([os.path.join(label_folder, lbl) for lbl in labels if os.path.exists(os.path.join(label_folder, lbl))])

# Split into train (70%), temp (30%)
train_images, temp_images, train_labels, temp_labels = train_test_split(
    all_images, all_labels, test_size=0.3, random_state=42
)

# Further split temp into val (15%) and test (15%)
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42
)

# Function to copy files to destination
def copy_files(src_list, dest_folder):
    for src_file in src_list:
        shutil.copy(src_file, dest_folder)

# Copy train files
copy_files(train_images, os.path.join(output_path, "train/images"))
copy_files(train_labels, os.path.join(output_path, "train/labels"))

# Copy val files
copy_files(val_images, os.path.join(output_path, "val/images"))
copy_files(val_labels, os.path.join(output_path, "val/labels"))

# Copy test files
copy_files(test_images, os.path.join(output_path, "test/images"))
copy_files(test_labels, os.path.join(output_path, "test/labels"))

print("Dataset successfully split into train, val, and test!")


Dataset successfully split into train, val, and test!


In [ ]:
import os

# Define paths
base_path = "/content/drive/MyDrive/Dataset_Split_for_yolo/"  # Base dataset directory

# Function to count files in a directory
def count_files(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# Initialize splits
splits = ["train", "val", "test"]
summary = {}

# Count files in each split
for split in splits:
    images_path = os.path.join(base_path, split, "images")
    labels_path = os.path.join(base_path, split, "labels")
    summary[split] = {
        "images": count_files(images_path),
        "labels": count_files(labels_path),
    }

# Print summary
print("\nDataset Summary:")
for split, counts in summary.items():
    print(f"{split.capitalize()}: {counts['images']} images, {counts['labels']} labels")



Dataset Summary:
Train: 2105 images, 2104 labels
Val: 453 images, 453 labels
Test: 453 images, 453 labels


In [ ]:
pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 20.7 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt', 'yolov8s.pt', etc., if needed

# Train the model
model.train(
    data='/content/drive/MyDrive/dataset1.yaml',  # Path to your dataset YAML file
    epochs=10,  # Number of epochs
    imgsz=[512, 384],  # Image size (matches your dataset dimensions)
    batch=16,  # Batch size; adjust based on your hardware
    device=0,  # Use 0 for GPU, or 'cpu' if GPU is unavailable
    project='/content/drive/MyDrive/Dataset_Split_for_yolo',  # Directory to save results

)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 105MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset1.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=[512, 384], save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Dataset_Split_for_yolo, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

100%|██████████| 755k/755k [00:00<00:00, 19.9MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        453        464       0.65      0.658      0.679      0.405



10 epochs completed in 0.134 hours.
Optimizer stripped from /content/drive/MyDrive/Dataset_Split_for_yolo/train2/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Dataset_Split_for_yolo/train2/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Dataset_Split_for_yolo/train2/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  1.94it/s]


                   all        453        464      0.647      0.658      0.676      0.405
                 Cloth        136        137      0.712       0.81      0.827      0.557
                 Glass        118        128       0.61      0.379      0.446      0.183
                 Paper         88         88      0.739      0.642      0.723       0.35
               Plastic        111        111      0.527      0.802      0.707       0.53
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to /content/drive/MyDrive/Dataset_Split_for_yolo/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b6a91296fe0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt', 'yolov8s.pt', etc., if needed

# Train the model
model.train(
    data='/content/drive/MyDrive/dataset1.yaml',  # Path to your dataset YAML file
    epochs=30,  # Number of epochs
    imgsz=[512, 384],  # Image size (matches your dataset dimensions)
    batch=16,  # Batch size; adjust based on your hardware
    device=0,  # Use 0 for GPU, or 'cpu' if GPU is unavailable
    project='/content/drive/MyDrive/Dataset_Split_for_yolo',  # Directory to save results

)

100%|██████████| 6.25M/6.25M [00:00<00:00, 101MB/s]


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset1.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=[512, 384], save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Dataset_Split_for_yolo, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_

100%|██████████| 755k/755k [00:00<00:00, 18.6MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 89.1MB/s]


...

                   all        453        464      0.751      0.727      0.776      0.458



30 epochs completed in 0.471 hours.
Optimizer stripped from /content/drive/MyDrive/Dataset_Split_for_yolo/train3/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Dataset_Split_for_yolo/train3/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Dataset_Split_for_yolo/train3/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  2.05it/s]


                   all        453        464      0.721      0.742      0.787      0.469
                 Cloth        136        137      0.827      0.781       0.87      0.606
                 Glass        118        128      0.613      0.594      0.604      0.242
                 Paper         88         88      0.785      0.747      0.841      0.398
               Plastic        111        111       0.66      0.847      0.834      0.629
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to /content/drive/MyDrive/Dataset_Split_for_yolo/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ab5583dd210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

**Training for 50 epochs**

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')  # Use 'yolov8n.pt', 'yolov8s.pt', etc., if needed

# Train the model
model.train(
    data='/content/drive/MyDrive/dataset1.yaml',  # Path to your dataset YAML file
    epochs=50,  # Number of epochs
    imgsz=[512, 384],  # Image size (matches your dataset dimensions)
    batch=16,  # Batch size; adjust based on your hardware
    device=0,  # Use 0 for GPU, or 'cpu' if GPU is unavailable
    project='/content/drive/MyDrive/Dataset_Split_for_yolo',  # Directory to save results

)

100%|██████████| 6.25M/6.25M [00:00<00:00, 106MB/s]

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/dataset1.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=[512, 384], save=True, save_period=-1, cache=False, device=0, workers=8, project=/content/drive/MyDrive/Dataset_Split_for_yolo, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, fo

100%|██████████| 755k/755k [00:00<00:00, 18.1MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

                   all        453        464        0.8      0.719      0.792      0.479



50 epochs completed in 0.791 hours.
Optimizer stripped from /content/drive/MyDrive/Dataset_Split_for_yolo/train4/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/Dataset_Split_for_yolo/train4/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/Dataset_Split_for_yolo/train4/weights/best.pt...
Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:07<00:00,  2.03it/s]


                   all        453        464        0.8      0.719      0.792      0.479
                 Cloth        136        137      0.881      0.809      0.881      0.619
                 Glass        118        128      0.693      0.562      0.614      0.256
                 Paper         88         88      0.838      0.739      0.855      0.405
               Plastic        111        111      0.789      0.766      0.817      0.637
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to /content/drive/MyDrive/Dataset_Split_for_yolo/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d934f9b7130>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

Object detection in image using model trained for 10 **epochs**

In [ ]:
import cv2
from google.colab import files
from ultralytics import YOLO
import shutil
import os

# Step 1: Upload image
uploaded = files.upload()  # This will prompt you to upload a file manually from your local system

# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/Dataset_Split_for_yolo/train2/weights/best.pt')  # Adjust path

# Step 2: Process the uploaded image
for img_name in uploaded.keys():
    # Read the uploaded image using OpenCV
    img_path = '/content/' + img_name  # The uploaded file will be in the /content folder
    img = cv2.imread(img_path)

    # Run the prediction
    results = model.predict(source=img_path, save=False)  # Don't save the result automatically yet

    # Get the predictions for the image
    prediction = results[0]

    # Step 3: Draw rectangles and labels around detected objects
    for box, conf, cls in zip(prediction.boxes.xyxy, prediction.boxes.conf, prediction.boxes.cls):
        # Extract the coordinates of the bounding box
        x1, y1, x2, y2 = map(int, box)  # Convert to integers for pixel coordinates

        # Get the class label
        class_label = prediction.names[int(cls)]  # Get class name from predicted index

        # Draw the rectangle around the object
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle

        # Add the class label above the rectangle
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, class_label, (x1, y1 - 10), font, 0.9, (0, 255, 0), 2, cv2.LINE_AA)  # Green label

    # Step 4: Save the annotated image to Google Drive
    save_dir = '/content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Save the annotated image
    output_path = os.path.join(save_dir, f"annotated_{img_name}")
    cv2.imwrite(output_path, img)  # Save the image to the specified directory

    # Display saved result path
    print(f"Annotated image saved as: {output_path}")

# Optional: If you want to download the result back to your local machine
# files.download(output_path)


Saving i15.jpg to i15.jpg

image 1/1 /content/i15.jpg: 384x512 (no detections), 10.8ms
Speed: 2.8ms preprocess, 10.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 512)
Annotated image saved as: /content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/annotated_i15.jpg


**Object detection in image using model trained for 30 epochs**

In [ ]:
import cv2
from google.colab import files
from ultralytics import YOLO
import shutil
import os

# Step 1: Upload image
uploaded = files.upload()  # This will prompt you to upload a file manually from your local system

# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/Dataset_Split_for_yolo/train3/weights/best.pt')  # Adjust path

# Step 2: Process the uploaded image
for img_name in uploaded.keys():
    # Read the uploaded image using OpenCV
    img_path = '/content/' + img_name  # The uploaded file will be in the /content folder
    img = cv2.imread(img_path)

    # Run the prediction
    results = model.predict(source=img_path, save=False)  # Don't save the result automatically yet

    # Get the predictions for the image
    prediction = results[0]

    # Step 3: Draw rectangles and labels around detected objects
    for box, conf, cls in zip(prediction.boxes.xyxy, prediction.boxes.conf, prediction.boxes.cls):
        # Extract the coordinates of the bounding box
        x1, y1, x2, y2 = map(int, box)  # Convert to integers for pixel coordinates

        # Get the class label
        class_label = prediction.names[int(cls)]  # Get class name from predicted index

        # Draw the rectangle around the object
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle

        # Add the class label above the rectangle
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, class_label, (x1, y1 - 10), font, 0.9, (0, 255, 0), 2, cv2.LINE_AA)  # Green label

    # Step 4: Save the annotated image to Google Drive
    save_dir = '/content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Save the annotated image
    output_path = os.path.join(save_dir, f"annotated_{img_name}")
    cv2.imwrite(output_path, img)  # Save the image to the specified directory

    # Display saved result path
    print(f"Annotated image saved as: {output_path}")

# Optional: If you want to download the result back to your local machine
# files.download(output_path)


Saving detect multi cat3.1.jpg to detect multi cat3.1 (2).jpg

image 1/1 /content/detect multi cat3.1 (2).jpg: 512x512 1 Paper, 1 Plastic, 7.8ms
Speed: 3.7ms preprocess, 7.8ms inference, 1.6ms postprocess per image at shape (1, 3, 512, 512)
Annotated image saved as: /content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/annotated_detect multi cat3.1 (2).jpg


With Confidence Score

In [ ]:
import cv2
from google.colab import files
from ultralytics import YOLO
import shutil
import os

# Step 1: Upload image
uploaded = files.upload()  # This will prompt you to upload a file manually from your local system

# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/Dataset_Split_for_yolo/train4/weights/best.pt')  # Adjust path

# Step 2: Process the uploaded image
for img_name in uploaded.keys():
    # Read the uploaded image using OpenCV
    img_path = '/content/' + img_name  # The uploaded file will be in the /content folder
    img = cv2.imread(img_path)

    # Run the prediction
    results = model.predict(source=img_path, save=False)  # Don't save the result automatically yet

    # Get the predictions for the image
    prediction = results[0]

    # Step 3: Draw rectangles and labels around detected objects
    for box, conf, cls in zip(prediction.boxes.xyxy, prediction.boxes.conf, prediction.boxes.cls):
        # Extract the coordinates of the bounding box
        x1, y1, x2, y2 = map(int, box)  # Convert to integers for pixel coordinates

        # Get the class label and confidence score
        class_label = prediction.names[int(cls)]  # Get class name from predicted index
        confidence = conf.item()  # Get confidence score as a float

        # Create the label text with the confidence score
        label = f"{class_label} {confidence:.2f}"  # Example: "Plastic 0.87"

        # Draw the rectangle around the object
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle

        # Add the class label with confidence above the rectangle
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, label, (x1, y1 - 10), font, 0.9, (0, 255, 0), 2, cv2.LINE_AA)  # Green label

    # Step 4: Save the annotated image to Google Drive
    save_dir = '/content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/'
    os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

    # Save the annotated image
    output_path = os.path.join(save_dir, f"annotated_{img_name}")
    cv2.imwrite(output_path, img)  # Save the image to the specified directory

    # Display saved result path
    print(f"Annotated image saved as: {output_path}")

# Optional: If you want to download the result back to your local machine
# files.download(output_path)


Saving detect multi cat3.jpg to detect multi cat3 (2).jpg

image 1/1 /content/detect multi cat3 (2).jpg: 512x512 1 Paper, 1 Plastic, 9.9ms
Speed: 2.7ms preprocess, 9.9ms inference, 1.5ms postprocess per image at shape (1, 3, 512, 512)
Annotated image saved as: /content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/annotated_detect multi cat3 (2).jpg


**Detection on video**

In [ ]:
import cv2
from google.colab import files
from ultralytics import YOLO
import os

# Step 1: Upload video
uploaded = files.upload()  # Upload a video file

# Load the trained YOLO model
model = YOLO('/content/drive/MyDrive/Dataset_Split_for_yolo/train3/weights/best.pt')  # Adjust path

# Step 2: Process the uploaded video
for video_name in uploaded.keys():
    # Video input path
    video_path = '/content/' + video_name  # Uploaded video is stored in the /content directory

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video
    fps = int(cap.get(cv2.CAP_PROP_FPS))  # Frames per second
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))  # Video frame width
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Video frame height

    # Output video path
    output_video_path = '/content/drive/MyDrive/Dataset_Split_for_yolo/annotated_results/annotated_' + video_name
    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)  # Ensure the directory exists

    # Initialize video writer
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # Process each frame of the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  # Exit loop if no more frames

        # Run YOLO prediction on the frame
        results = model.predict(source=frame, save=False)  # Predict on the frame

        # Get the prediction for the frame
        prediction = results[0]

        # Draw bounding boxes and labels on the frame
        for box, conf, cls in zip(prediction.boxes.xyxy, prediction.boxes.conf, prediction.boxes.cls):
            # Extract the coordinates of the bounding box
            x1, y1, x2, y2 = map(int, box)  # Convert to integers for pixel coordinates

            # Get the class label
            class_label = prediction.names[int(cls)]  # Get class name from predicted index

            # Draw the rectangle around the object
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green rectangle

            # Add the class label above the rectangle
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, class_label, (x1, y1 - 10), font, 0.9, (0, 255, 0), 2, cv2.LINE_AA)  # Green label

        # Write the annotated frame to the output video
        out.write(frame)

    # Release resources
    cap.release()
    out.release()

    # Display the saved video path
    print(f"Annotated video saved as: {output_video_path}")

# Optional:  to download the result back to your local machine
# files.download(output_video_path)



Saving Detection video3.mp4 to Detection video3.mp4

0: 320x512 1 Paper, 46.5ms
Speed: 1.5ms preprocess, 46.5ms inference, 1.3ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 1 Paper, 8.8ms
Speed: 1.9ms preprocess, 8.8ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 (no detections), 7.4ms
Speed: 1.7ms preprocess, 7.4ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 1 Plastic, 6.3ms
Speed: 1.7ms preprocess, 6.3ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 1 Paper, 1 Plastic, 14.9ms
Speed: 1.8ms preprocess, 14.9ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 (no detections), 11.2ms
Speed: 1.8ms preprocess, 11.2ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 (no detections), 11.1ms
Speed: 2.6ms preprocess, 11.1ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 512)

0: 320x512 (no detections), 10.8ms
Speed: